# IMPORT DES LIBRAIRIES

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

# OPTIONS PANDAS 

In [2]:
pd.options.display.float_format = '{:.0f}'.format

# OUVERTURE DE NOTRE DATASET

In [3]:
df = pd.read_csv('../DATA/OUT/dataset_hors_outliers_surfaces_and_piece.csv')
df.head()

,Prix,DT_Date,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,NoDisp,NoVoie,BTQ,...,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat,ID,PrixTerrainM2,PrixBatiM2,PrixCarrezM2
0,165000,2019-01-04,970,70,0,4,0,1,19,NaN,...,"[150000.0, 200000.0)",1,2019,1,True,"(60.0, 70.0]",1,170,2357,0
1,509100,2019-01-09,821,193,0,6,0,1,5,NaN,...,"[500000.0, 600000.0)",1,2019,1,True,"(144.0, 1000.0]",2,620,2638,0
2,80000,2019-01-09,0,16,15,1,1,1,33,NaN,...,"[50000.0, 100000.0)",1,2019,1,False,"(0.999, 35.0]",3,0,5000,5239
3,382700,2019-01-10,1428,129,0,4,0,2,8,B,...,"[300000.0, 400000.0)",1,2019,1,True,"(117.0, 144.0]",4,268,2967,0
4,421000,2019-01-08,337,136,0,6,0,1,52,NaN,...,"[400000.0, 500000.0)",1,2019,1,True,"(117.0, 144.0]",5,1249,3096,0


# SUPPRESSION DES FEATURES NON ESSENTIELLES

In [4]:
# Préparation de df_model en excluant les colonnes spécifiées dans votre code initial
columns_to_drop = ['NoDisp', 'NoVoie', 'BTQ', 'TypeVoie', 'CodeVoie', 'Voie', 'CommuneName', 'NombreLots', 'NatureCultureName', 'NatureCultureSpecialeName',
                   'slice', 'quarter', 'Month', 'DT_Date', 'SurfaceBatiCat', 'ID',
                   'PrixTerrainM2', 'PrixBatiM2', 'PrixCarrezM2', 'AvecTerrain']

# Créer df_model
df_model = df.drop(columns=columns_to_drop)

# Vérifier s'il reste des colonnes non numériques qui doivent être exclues ou traitées séparément
# df_model.select_dtypes(exclude=['number']).columns.tolist()

df_model = df_model.dropna(subset=['Prix', 'SurfaceBati', 'CodePostal'])
df_model.isna().sum()

Prix                       0
SurfaceTerrain             0
SurfaceBati                0
SurfaceCarrez              0
NombrePiecesPrincipales    0
CodePostal                 0
TypeLocalName              0
Year                       0
dtype: int64

In [5]:
df_model.shape

(159545, 8)

In [6]:
df_model

,Prix,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombrePiecesPrincipales,CodePostal,TypeLocalName,Year
0,165000,970,70,0,0,33480,Maison,2019
1,509100,821,193,0,0,33320,Maison,2019
2,80000,0,16,15,1,33000,Appartement,2019
3,382700,1428,129,0,0,33160,Maison,2019
4,421000,337,136,0,0,33300,Maison,2019
...,...,...,...,...,...,...,...,...
159540,160000,273,50,0,0,33990,Maison,2023
159541,190000,1349,95,0,0,33540,Maison,2023
159542,190000,500,95,0,0,33540,Maison,2023
159543,288654,944,95,0,0,33440,Maison,2023


# TRANSFORMATION DE LOCALISATION EN NUMERIQUE (ONE HOT ENCODER)

In [7]:
# Assurez-vous qu'il n'y a pas de valeurs NaN dans 'CP' avant le One-Hot Encoding
df_model = df_model.dropna(subset=['CodePostal'])

# Réinitialiser l'index pour s'assurer que l'alignement est conservé après la suppression des NaN
df_model.reset_index(drop=True, inplace=True)

# Encoder les colonnes 'CodePostal' et 'TypeLocalName' avec OneHotEncoder
categorical_cols = ['CodePostal', 'TypeLocalName']
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_model[categorical_cols])

# Convertir les résultats encodés en DataFrame
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Réinitialiser l'index sur le DataFrame encodé pour garantir l'alignement lors de la fusion
encoded_df.reset_index(drop=True, inplace=True)

# Fusionner le DataFrame encodé avec le df_model original en s'assurant que les indices sont alignés
df_model = pd.concat([df_model, encoded_df], axis=1)

# Supprimer les colonnes catégorielles originales
df_model = df_model.drop(columns=categorical_cols)

In [8]:
df_model

,Prix,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombrePiecesPrincipales,Year,CodePostal_33000,CodePostal_33100,CodePostal_33110,CodePostal_33112,...,CodePostal_33890,CodePostal_33910,CodePostal_33920,CodePostal_33930,CodePostal_33950,CodePostal_33970,CodePostal_33980,CodePostal_33990,TypeLocalName_Appartement,TypeLocalName_Maison
0,165000,970,70,0,0,2019,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,509100,821,193,0,0,2019,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,80000,0,16,15,1,2019,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,382700,1428,129,0,0,2019,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,421000,337,136,0,0,2019,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159540,160000,273,50,0,0,2023,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
159541,190000,1349,95,0,0,2023,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
159542,190000,500,95,0,0,2023,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
159543,288654,944,95,0,0,2023,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# SEPARATION DU JEU DE DONNES

In [9]:
# Séparation du jeu de données en X (caractéristiques) et y (cible)
X, y = df_model.drop(columns=["Prix"]), df_model["Prix"]

# Séparation en ensembles de train, de test, et de validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.10, random_state=42)

In [10]:
# Instancier le RobustScaler
scaler = RobustScaler()

# Sélectionner les colonnes numériques dans X_train pour l'application du RobustScaler
num_cols = X_train.select_dtypes(include=['number']).columns

# Appliquer le RobustScaler uniquement sur les colonnes numériques de l'ensemble d'entraînement
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Utiliser le scaler ajusté pour transformer les ensembles de test et de validation
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])

print(f'X_train : {X_train.shape[0]}, X_test : {X_test.shape[0]}, X_val : {X_val.shape[0]}')

X_train : 106895, X_test : 47385, X_val : 5265


In [11]:
# Étape 1: Instancier le modèle
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [13]:
# Étape 2: Entraîner le modèle
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
# Étape 3: Prédire les valeurs sur l'ensemble de test
y_pred = rf.predict(X_test)

# Étape 4: Évaluer les performances
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {round(r2 * 100, 2)} % du prix est expliqué par nos features actuelles")

Mean Squared Error: 11278613216.023682
Mean Absolute Error: 65509.68363187109
R-squared: 68.16 % du prix est expliqué par nos features actuelles
